In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from aml_obs.drift import Drift_Analysis
from aml_obs.collector import Online_Collector
from aml_obs.query import RT_Visualization
from azureml.core import Workspace
import pandas as pd
from azureml.core.model import Model
from sklearn.preprocessing import LabelEncoder
import joblib

ws = Workspace.from_config()


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

### Real time metrics logging and visualization
#### You need to run this using your local computer or if you run with Azure ML Compute Instance, please do so from VSCode as Dash visualization requires access to localhost

In [ ]:
# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('today'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging_daemon()

In [ ]:
# from monitoring.query import RT_Visualization
streaming_table_name="streaming_test"
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Model Monitoring

The deploy_model utility install AML CLI v2, create managed online endpoint, deploy the model. These steps are optional incase you want to supply your own names and objects. Check the function to see all paramters. The below just create everything.

In [5]:
from deployment.deploy_util import deploy_model,score_test

scoring_uri, scoring_key = deploy_model(ws=ws)

Install ML CLI v2
Creating managed online endpoint  
creating online endpoint with name  iris-ep6667
az ml online-endpoint create -f deployment/endpoint.yml --name iris-ep6667
Deploy ML model 
deployment is done


Once you have the model deployed with the collector object inside, you can just call it to simulate data creation. Data is collected to ADX table nIRIS_MODEL.

In [6]:
import time
for _ in range(1000):
    time.sleep(3)
    score_test(scoring_uri, scoring_key)

b'["setosa", "versicolor", "setosa", "virginica", "setosa", "versicolor", "virginica", "virginica", "versicolor", "versicolor"]'
b'["virginica", "setosa", "setosa", "setosa", "virginica", "versicolor", "versicolor", "setosa", "setosa", "versicolor"]'


### Monitoring Dashboard

Now that data is collected. Go to ADX Dashboard and import the quick_start/ADX_dashboards/databoard_Model_Monitoring.json to create a monitoring dashboard for your model.

Review https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file for instruction on how to import the dashboard.

You may need to correct a few default paramters such as connection to make dashboard work